<a href="https://colab.research.google.com/github/radr44/Iterative_Convolutional_Neural_Network/blob/master/checkpoints/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import numpy as np
import cv2 as cv
import subsampling
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import ICNN
import preprocessing
import copy

In [0]:
data_transforms = transforms.Compose([
        transforms.Resize((96,144)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor() ])

In [0]:
data_dir = '/content/drive/My Drive/Superset'
image_dataset = datasets.ImageFolder(data_dir,data_transforms)
class_names = image_dataset.classes
train_set, hold_set = torch.utils.data.random_split(image_dataset, [23582,10000])
train_set, throw_set = torch.utils.data.random_split(train_set, [23580,2] )
val_set, test_set = torch.utils.data.random_split(hold_set, [5000,5000])
dataset_train = torch.utils.data.DataLoader(train_set, batch_size = 5, shuffle = True, num_workers = 4)
dataset_val = torch.utils.data.DataLoader(val_set, batch_size = 5, shuffle = True, num_workers = 4)
dataset_test = torch.utils.data.DataLoader(test_set, batch_size = 5, shuffle = True, num_workers = 4)

In [0]:
def train_model_p1(model, criterion, optimizer, scheduler, num_epochs=10):
  #train whole network using the fc layers of last ucnn
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'dev']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for batch_idx,(inputs, labels) in enumerate(dataset_combined[phase]):
               
               
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                if batch_idx % 50 == 0:
                  print('Epoch : {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Accuracy:{:.3f}%'.format(
                    epoch, batch_idx*len(inputs), len(dataset_combined["train"].dataset), 100*batch_idx / len(dataset_train), loss.data, float(running_corrects*100) / float(5*(batch_idx+1))))
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'dev' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    print(scheduler)

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [0]:
dataset_combined = {"train":dataset_train,"dev":dataset_val}

In [0]:
ucnn_1 = ICNN.micro_cnn_Alexnet(k=0)
ucnn_2 = ICNN.micro_cnn_Alexnet(k=1)
ucnn_3 = ICNN.micro_cnn_Alexnet(k=2)
ucnn_4 = ICNN.micro_cnn_Alexnet(k=3)
ucnn_5 = ICNN.micro_cnn_Alexnet(k=4)
ucnn_6 = ICNN.micro_cnn_Alexnet(k=5)
ucnn_7 = ICNN.micro_cnn_Alexnet(k=6)
ucnn_8 = ICNN.micro_cnn_Alexnet(k=7)
ucnn_9 = ICNN.micro_cnn_Alexnet(k=8)
ucnn_10 = ICNN.micro_cnn_Alexnet(k=9,nfc=512)
ucnn = [ucnn_1,ucnn_2,ucnn_3,ucnn_4,ucnn_5,ucnn_6]

In [0]:
icnn=ICNN.ICNN_Alexnet(ucnn=ucnn)
criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.Adam(icnn.parameters(),lr=0.0001)

exp_lr_scheduler = lr_scheduler.MultiplicativeLR(optimizer_ft,lr_lambda=0.9)

In [0]:
def fit(model, train_loader):
    optimizer = torch.optim.Adam(model.parameters(),lr=0.002)
    error = nn.CrossEntropyLoss()
    EPOCHS = 6
    model.train()
    for epoch in range(EPOCHS):
        correct = 0
        for batch_idx, (x_batch, y_batch) in enumerate(train_loader):
            var_x_batch = Variable(x_batch).float()
            var_y_batch = Variable(y_batch)
            optimizer.zero_grad()
            output = model(var_x_batch)
            loss = error(output, var_y_batch)
            loss.backward()
            optimizer.step()

            # Total correct predictions
            predicted = torch.max(output.data, 1)[1] 
            correct += (predicted == var_y_batch).sum()
            #print(correct)
            if batch_idx % 50 == 0:
                print('Epoch : {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Accuracy:{:.3f}%'.format(
                    epoch+1, batch_idx*len(x_batch), len(train_loader.dataset), 100*batch_idx / len(train_loader), loss.data, float(correct*100) / float(4*(batch_idx+1))))

In [54]:
len(list(icnn.parameters()))

96

In [10]:
model_ft = train_model_p1(icnn, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=3)

Epoch 0/9
----------
Epoch : 0 [0/23582 (0%)]	Loss: 2.675113	 Accuracy:25.000%
Epoch : 0 [200/23582 (1%)]	Loss: 2.911409	 Accuracy:8.824%
Epoch : 0 [400/23582 (2%)]	Loss: 2.572189	 Accuracy:11.139%
Epoch : 0 [600/23582 (3%)]	Loss: 2.478086	 Accuracy:12.583%
Epoch : 0 [800/23582 (3%)]	Loss: 3.168175	 Accuracy:17.164%
Epoch : 0 [1000/23582 (4%)]	Loss: 1.265169	 Accuracy:20.916%
Epoch : 0 [1200/23582 (5%)]	Loss: 1.787857	 Accuracy:25.166%
Epoch : 0 [1400/23582 (6%)]	Loss: 1.192203	 Accuracy:28.205%
Epoch : 0 [1600/23582 (7%)]	Loss: 1.167319	 Accuracy:32.232%
Epoch : 0 [1800/23582 (8%)]	Loss: 1.968131	 Accuracy:34.701%
Epoch : 0 [2000/23582 (8%)]	Loss: 2.150630	 Accuracy:37.375%
Epoch : 0 [2200/23582 (9%)]	Loss: 0.823149	 Accuracy:39.746%
Epoch : 0 [2400/23582 (10%)]	Loss: 0.853007	 Accuracy:41.597%
Epoch : 0 [2600/23582 (11%)]	Loss: 2.608127	 Accuracy:43.625%
Epoch : 0 [2800/23582 (12%)]	Loss: 1.099730	 Accuracy:45.292%
Epoch : 0 [3000/23582 (13%)]	Loss: 0.647802	 Accuracy:47.204%
Epoch :

IndexError: ignored

In [52]:
fit(icnn,dataset_val)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:432: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 15])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: ignored

In [0]:
from torch.autograd import Variable

In [0]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(icnn)    

9970890

In [0]:
a=torch.tensor([[1,2,3,4],[5,6,7,8],[8,9,5,1]])
torch.max(a,dim=1)


torch.return_types.max(values=tensor([4, 8, 9]), indices=tensor([3, 3, 1]))

In [0]:
icnn(torch.ones((4,3,96,144))).size()

torch.Size([4, 15])

In [0]:
icnn.state_dict

<bound method Module.state_dict of ICNN_Alexnet(
  (ucnn): ModuleList(
    (0): micro_cnn_Alexnet(
      (conv1b): Conv2d(3, 18, kernel_size=(11, 11), stride=(1, 1), padding=(5, 5))
      (conv2): Conv2d(18, 48, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (conv3): Conv2d(48, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv4): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv5): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (fc1): Linear(in_features=11040, out_features=512, bias=True)
      (fc2): Linear(in_features=512, out_features=256, bias=True)
      (fc3): Linear(in_features=256, out_features=15, bias=True)
    )
    (1): micro_cnn_Alexnet(
      (conv1b): Conv2d(3, 18, kernel_size=(11, 11), stride=(1, 1), padding=(5, 5))
      (conv2): Conv2d(18, 48, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (conv3): Conv2d(48, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv4): C